In [4]:
import pandas as pd
import numpy as np
import chardet
import os

# ------------------------- 改进的编码检测函数 -------------------------
def get_file_encoding(file_path):
    """更可靠的编码检测函数"""
    with open(file_path, 'rb') as f:
        raw_data = f.read(100000)  # 读取更多字节以提高准确性
        result = chardet.detect(raw_data)
        return result['encoding'] if result['confidence'] > 0.7 else None

# ------------------------- 1. 合并数据（改进版本） -------------------------
file_paths = [
    r"E:\2015年国内主要城市年度数据.csv",
    r"E:\2016年国内主要城市年度数据.csv",
    r"E:\2017年国内主要城市年度数据.csv"
]

dfs = []
success_count = 0

for path in file_paths:
    if not os.path.exists(path):
        print(f"文件不存在: {path}")
        continue
    
    print(f"\n处理文件: {path}")
    
    # 尝试多种编码方式
    encodings_to_try = ['utf-8-sig', 'gb18030', 'latin1', 'iso-8859-1', 'cp1252']
    
    for encoding in encodings_to_try:
        try:
            print(f"尝试编码: {encoding}")
            df = pd.read_csv(
                path, 
                encoding=encoding,
                on_bad_lines='skip',
                engine='python'  # 使用Python引擎更灵活
            )
            
            # 获取年份（从文件名中提取）
            filename = os.path.basename(path)
            year_str = filename.split("年")[0]
            if year_str.isdigit():
                df["年份"] = int(year_str)
            else:
                print(f"警告: 无法从文件名 '{filename}' 中提取年份")
                continue
                
            dfs.append(df)
            success_count += 1
            print(f"成功读取 {path}，使用编码 {encoding}，数据量 {len(df)} 行")
            break  # 成功读取后跳出编码尝试循环
            
        except Exception as e:
            print(f"编码 {encoding} 失败: {str(e)[:100]}...")
    
    # 如果所有编码都失败，尝试自动检测
    if not any(encoding in encodings_to_try for encoding in encodings_to_try):
        print("所有预设编码失败，尝试自动检测...")
        auto_enc = get_file_encoding(path)
        if auto_enc:
            try:
                df = pd.read_csv(path, encoding=auto_enc, on_bad_lines='skip', engine='python')
                # 添加年份列
                filename = os.path.basename(path)
                year_str = filename.split("年")[0]
                if year_str.isdigit():
                    df["年份"] = int(year_str)
                else:
                    print(f"警告: 无法从文件名 '{filename}' 中提取年份")
                    continue
                
                dfs.append(df)
                success_count += 1
                print(f"成功读取 {path}，使用自动检测编码 {auto_enc}，数据量 {len(df)} 行")
            except Exception as e:
                print(f"自动检测编码 {auto_enc} 失败: {str(e)[:100]}...")
        else:
            print("无法自动检测文件编码")

if success_count == 0:
    print("\n错误: 所有文件均无法读取！请检查文件路径和格式")
    print("建议: 使用文本编辑器打开CSV文件，查看其实际编码格式")
    exit()

# 合并所有成功读取的数据
merged_df = pd.concat(dfs, ignore_index=True)

# 打印列名以便调试
print("\n数据列名:", merged_df.columns.tolist())

# 重命名可能的列名变体
column_mapping = {
    'gdp': 'GDP',
    '地区生产总值': 'GDP',
    '医院数': '医院、卫生院数',
    '医疗卫生机构数': '医院、卫生院数',
    '社会消费品零售总额': '社会商品零售总额',
    '零售总额': '社会商品零售总额'
}

# 应用列名标准化
merged_df.rename(columns=column_mapping, inplace=True)

# ------------------------- 2. 处理缺省值 -------------------------
# 只处理数值列
numeric_cols = merged_df.select_dtypes(include=np.number).columns
merged_df[numeric_cols] = merged_df[numeric_cols].fillna(0)

# ------------------------- 3. 按年份聚合GDP总和 -------------------------
if "GDP" in merged_df.columns and "年份" in merged_df.columns:
    gdp_by_year = merged_df.groupby("年份")["GDP"].sum()
    print("\n每年国内生产总值总和：")
    print(gdp_by_year)
else:
    print("警告: 缺少 'GDP' 或 '年份' 列，无法计算GDP总和")

# ------------------------- 4. 计算城市GDP年均增长率 -------------------------
if "城市" in merged_df.columns and "GDP" in merged_df.columns and "年份" in merged_df.columns:
    # 创建透视表
    pivot_df = merged_df.pivot_table(
        index="城市", 
        columns="年份", 
        values="GDP",
        aggfunc='mean'
    )
    
    # 确保有2015和2017年数据
    if 2015 in pivot_df.columns and 2017 in pivot_df.columns:
        # 计算年均增长率
        pivot_df["年均增长率(%)"] = ((pivot_df[2017] / pivot_df[2015]) ** (1/2) - 1) * 100
        
        # 处理无穷大和NaN值
        pivot_df.replace([np.inf, -np.inf], np.nan, inplace=True)
        pivot_df["年均增长率(%)"].fillna(0, inplace=True)
        
        # 获取最高和最低增长率
        top_5 = pivot_df["年均增长率(%)"].nlargest(5)
        bottom_5 = pivot_df["年均增长率(%)"].nsmallest(5)
        
        print("\nGDP 年均增长率最高的5个城市：")
        print(top_5)
        print("\nGDP 年均增长率最低的5个城市：")
        print(bottom_5)
    else:
        print("警告: 缺少2015年或2017年数据，无法计算增长率")
else:
    print("警告: 缺少关键列（城市、GDP、年份），无法计算增长率")

# ------------------------- 5. 医院数量归一化 -------------------------
hospital_col = "医院、卫生院数"
if hospital_col in merged_df.columns and "年份" in merged_df.columns:
    # 创建新列存储归一化结果
    merged_df[f"{hospital_col}_归一化"] = 0
    
    for year in merged_df["年份"].unique():
        year_mask = merged_df["年份"] == year
        year_df = merged_df[year_mask]
        
        if len(year_df) == 0:
            continue
            
        min_val = year_df[hospital_col].min()
        max_val = year_df[hospital_col].max()
        
        # 避免除以零
        if max_val > min_val:
            merged_df.loc[year_mask, f"{hospital_col}_归一化"] = (
                (year_df[hospital_col] - min_val) / (max_val - min_val)
                print(f"\n医院数量归一化完成，示例数据：")
    print(merged_df[["城市", "年份", hospital_col, f"{hospital_col}_归一化"]].head())
                
    
    print(f"\n医院数量归一化完成，示例数据：")
    print(merged_df[["城市", "年份", hospital_col, f"{hospital_col}_归一化"]].head())
else:
    print(f"警告: 缺少 '{hospital_col}' 列或 '年份' 列，无法进行归一化")

# ------------------------- 6. 提取四大城市数据 -------------------------
cities = ["北京", "上海", "广州", "深圳"]
columns_to_extract = ["城市", "年份", "GDP", "社会商品零售总额"]

# 确保列存在
available_cols = [col for col in columns_to_extract if col in merged_df.columns]

if len(available_cols) > 1 and "城市" in available_cols and "年份" in available_cols:
    # 提取数据
    selected_df = merged_df[
        (merged_df["城市"].isin(cities)) & 
        (merged_df["年份"].between(2015, 2017))
    ][available_cols]
    
    if not selected_df.empty:
        save_path = r"E:\北上广深_2015-2017_GDP_零售数据.csv"
        selected_df.to_csv(save_path, index=False, encoding='utf-8-sig')
        print(f"\n四大城市数据已保存至: {save_path}")
        print(f"包含列: {available_cols}")
    else:
        print("警告: 没有找到符合条件的四大城市数据")
else:
    print("警告: 缺少必要的列来提取四大城市数据")

# ------------------------- 最终数据保存 -------------------------
# 保存处理后的完整数据
merged_save_path = r"E:\合并后_城市数据.csv"
merged_df.to_csv(merged_save_path, index=False, encoding='utf-8-sig')
print(f"\n所有合并数据已保存至: {merged_save_path}")

SyntaxError: invalid syntax (1912694600.py, line 175)